*Created by Petteri Nevavuori <<petteri.nevavuori@gmail.com>>*

---

**CHIO & FREEMAN: MACHINE LEARNING & SECURITY (2018)** <br>
*<small>Otsikot kirjan mukaan, muutoin suomeksi.</small>*

<h1>Sisällysluettelo<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Mallinnusharjoitus" data-toc-modified-id="Mallinnusharjoitus-1">Mallinnusharjoitus</a></span><ul class="toc-item"><li><span><a href="#Datajoukosta" data-toc-modified-id="Datajoukosta-1.1">Datajoukosta</a></span></li><li><span><a href="#Mallinnuksesta" data-toc-modified-id="Mallinnuksesta-1.2">Mallinnuksesta</a></span><ul class="toc-item"><li><span><a href="#LSTM" data-toc-modified-id="LSTM-1.2.1">LSTM</a></span></li><li><span><a href="#Datan-käsittely" data-toc-modified-id="Datan-käsittely-1.2.2">Datan käsittely</a></span></li></ul></li><li><span><a href="#Datan-esikäsittely" data-toc-modified-id="Datan-esikäsittely-1.3">Datan esikäsittely</a></span><ul class="toc-item"><li><span><a href="#Datan-lataus" data-toc-modified-id="Datan-lataus-1.3.1">Datan lataus</a></span></li><li><span><a href="#Datan-muokkaus" data-toc-modified-id="Datan-muokkaus-1.3.2">Datan muokkaus</a></span></li><li><span><a href="#Nimiöity-datajoukko" data-toc-modified-id="Nimiöity-datajoukko-1.3.3">Nimiöity datajoukko</a></span></li></ul></li><li><span><a href="#Mallin-rakentaminen" data-toc-modified-id="Mallin-rakentaminen-1.4">Mallin rakentaminen</a></span><ul class="toc-item"><li><span><a href="#Dataset-luokka" data-toc-modified-id="Dataset-luokka-1.4.1">Dataset-luokka</a></span></li></ul></li></ul></li></ul></div>

# Mallinnusharjoitus

Tässä Notebookissa käydään läpi poikkeamien havaintomallin koulutus harjoitusmielessä. Ensin esitellään data, minkä jälkeen esitellään koulutettava malli perusajatuksineen ja tämän jälkeen koulutetaan itse malli.

## Datajoukosta

Datajoukkona toimii [ADFA-LD](https://www.unsw.adfa.edu.au/unsw-canberra-cyber/cybersecurity/ADFA-IDS-Datasets/), joka on vuonna 2012 alunperin koottu käyttöjärjestelmän toimintaan liittyvien poikkeamien havaintomallien koulutusta varten. Datajoukko koostuu tyystin järjestelmäkutsuista, jotka on enkoodattu numeeriseen muotoon. Yksi numero vastaa yhtä Ubuntun järjestelmäkutsuista. Data on kerätty Ubuntu-käyttöjärjestelmän versiosta 11.04 käyttämällä `auditd`-palvelua. Tärkein palvelun keräämä tieto on `syscall`-tietue, joka ADFA-LD:ssä on kerätty sekvensseinä. Raakamuodossaan `auditd`:n keräämä data näyttää seuraavalta:

    =====================================================================
    #  date         time        syscall pid     comm        auid    event
    =====================================================================
    1. 12/04/2018   14:14:35    257     2309    DOM Worker  1000    82
    2. 12/04/2018   14:16:58    257     2309    DOM Worker  1000    83
    3. 12/04/2018   14:21:53    257     2309    DOM Worker  1000    90
    4. 12/04/2018   14:26:58    257     2309    DOM Worker  1000    91
    5. 12/04/2018   14:29:28    257     2309    DOM Worker  1000    92
    6. 12/04/2018   14:31:58    257     2309    DOM Worker  1000    93
    7. 12/04/2018   14:36:58    257     2309    DOM Worker  1000    94
    8. 12/04/2018   14:40:00    257     2309    DOM Worker  1000    95
    9. 12/04/2018   14:41:58    257     2309    DOM Worker  1000    96
    ...

Datajoukossa data on seuraavassa muodossa:

    54 175 120 175 175 3 175 175 120 175 120 175 120 175 175 120 175 3 3 3 
    175 120 175 175 175 7 3 3 175 120 175 7 175 7 119 174 54 3 3 175 175 3 
    120 175 175 120 175 120 120 175 175 54 140 3 175 120 175 175 175 175 ...
Datajoukko on jaoteltu siten, että siinä on erilliset datajoukot normaalin perustilan oppimiselle, hyökkäyksille ja validointitarkoitukseen kerätylle normaalille datalle. Datajoukko on alunperin muodostettu siten, että siitä muodostetaan uniikkeja $n$-grammeja ja näiden avulla pyritään tunnistamaan perustilasta poikkeava toiminta. Se ei siis ole mitenkään ajallisesti riippuvaisessa muodossa, vaan koostuu yksittäisistä useasta järjestelmäkutsusta muodostuvista jäljistä (*trace*). Mallin koulutuksen päämääränä on tällöin oppia normaali perustila käyttämällä koulutukseen haitattomia jälkiä.

## Mallinnuksesta

Koska tavoitteena on oppia erottamaan poikkeava toiminta normaalista, on kyseessä yksinkertaistetusti binääriluokitteluongelma. Vaikka datajoukon jäljet ovatkin annettu toisistaan riippumattomina, ovat jälkien sisäiset järjestelmäkutsut vahvasti sekvenssisiä. 

### LSTM

Tästä syystä LSTM-malli voisi soveltua luokittimeksi siten, lukisi jäljet ikäänkuin rihmana tarkastellen aina muutamaa järjestelmäkutsua kerrallaan. Tärkeää kuitenkin on, ettei jälkiä sekoiteta keskenään ja mallille anneta valheellista käsitystä jälkien välisistä yhteyksistä. Koska jäljet koostuvat monista järjestelmäkutsuista (100+), voisi pinottu LSTM olla vain yksitasoista vastaavaa parempi - näin malli voisi paremmin oppia käsiteltävän sekvenssipituuden ylittäviä ajallisia piirteitä.

Vaikka LSTM-malli ei itsessään ole ns. tilaa oppiva, kuten vaikkapa RBM, testataan tässä harjoituksessa sitä, että voidaanko kyseistä mallia käyttää poikkeamien havaitsemiseen. Tätä tavoitellaan siten, että malli opetetaan tuottamaan ADFA-datajoukon koulutusosajoukolla luokittelutulokseksi arvo `1`. Koulutuksen jälkeen mallia arvioidaan validointi- ja hyökkäysjoukoilla. Arvioinnin tavoitteena on selvittää, tekeekö malli eroa hyökkäysekvenssien ja normaalien sekvenssien suhteen.

### Datan käsittely

Vaikkakin data on näennäisen numeerista, on kyseessä kuitenkin ennemmin lista kategorisia kuin jatkuvia arvoja. Tällöin data olisi syötettävä ennemmin harvana matriisina siten, että yksi sarake vastaa yhtä järjestelmäkutsua ja yhdellä sarakkeella esiintyy vain yksi kutsu. Tällöin data näyttäisi listan sijasta seuraavankaltaiselta:
    
    ==================================
    #. 1 2 3 4 5 6 ... 300 301 302 ...
    ==================================
    1. 0 0 1 0 0 0 ... 0   0   0   ...
    2. 0 0 1 0 0 0 ... 0   0   0   ...
    3. 0 0 1 0 0 0 ... 0   0   0   ...
    4. 0 1 0 0 0 0 ... 0   0   0   ...
    5. 0 0 0 0 0 0 ... 0   1   0   ...
    ...

## Datan esikäsittely

Ylätason esittelyn jälkeen on parempi siirtyä suoraan toteutukseen ja selittää tekemistä auki matkan varrella. Ensin aloitetaan datan käsittelyllä, minkä jälkeen jatketaan mallin ja siihen liittyvän datajoukkojen käsittelijöiden rakentamisella. Lopuksi malli koulututetaan ja sen tuloksia vertaillaan.

### Datan lataus

Aloitetaan datan käsittelyosio määrittämällä polut oikeisiin kansioihin.

In [1]:
import os

ADFA_LD_DIR = os.path.join(os.getcwd(),'datasets','ADFA-LD')
TRAINING_DIR = os.path.join(ADFA_LD_DIR,'Training_Data_Master')
ATTACK_DIR = os.path.join(ADFA_LD_DIR,'Attack_Data_Master')
VALIDATION_DIR = os.path.join(ADFA_LD_DIR,'Validation_Data_Master')

Kurkistetaanpa kansioiden sisältöihin.

In [2]:
print('Training-kansion sisältöä:')
print(f'  {", ".join(os.listdir(TRAINING_DIR)[:6])}, ...')

print('Attack-kansion sisältöä:')
print(f'  {", ".join(os.listdir(ATTACK_DIR)[:6])}, ...')

print('Validation-kansion sisältöä:')
print(f'  {", ".join(os.listdir(VALIDATION_DIR)[:6])}, ...')

Training- ja Validation-kansiot sisältävät suoraan tekstitiedostoja, jotka itsessään ovat järjestelmäkutsuja sisältäviä jälkiä. Katsotaanpa tarkemmin vielä Attack-kansion alikansioita.

In [3]:
print('Attack-kansion alikansioiden sisältöä:')
print(f'  {", ".join(os.listdir(os.path.join(ATTACK_DIR,os.listdir(ATTACK_DIR)[0]))[:3])}, ...')
print(f'  {", ".join(os.listdir(os.path.join(ATTACK_DIR,os.listdir(ATTACK_DIR)[1]))[:3])}, ...')
print(f'  {", ".join(os.listdir(os.path.join(ATTACK_DIR,os.listdir(ATTACK_DIR)[2]))[:3])}, ...')

Hyökkäystyypit on siis jaoteltu omiin kansioihinsa. Kokeillaanpa seuraavaksi yhden järjestelmäkutsuja sisältävän tiedoston käsittelyä.

In [4]:
content=[]
with open(os.path.join(TRAINING_DIR,os.listdir(TRAINING_DIR)[0])) as file:
    for line in file.readlines():
        content.append(line)

print(f'Ladatun tiedoston')
print(f'  rivimäärä: {len(content)}')
print(f'  sisällön tyyppi: {type(content[0])}')
print(f'  muutama järjestelmäkutsua: {content[0][:60]}')

Eli kutsujen numeeriset arvot on parsittava. Tehdäänpä vielä se.

In [5]:
with open(os.path.join(TRAINING_DIR,os.listdir(TRAINING_DIR)[0])) as file:
    for line in file.readlines():
        sequence = [int(syscall) for syscall in line.split()]
        break

print(f'Tiedoston sekvenssin')
print(f'  pituus: {len(sequence)}')
print(f'  sisällön tyyppi: {type(sequence[0])}')
print(f'  muutama järjestelmäkutsua: {sequence[:20]}')

Ladataanpa seuraavaksi omiin muuttujiinsa kaikki data. Training- ja Validation-datat voidaan ladata suoraan listoina. Vaikka hyökkäystyypit ovatkin lähtökohtaisesti eroteltuina, nekin imaistaan vain yhdeksi suureksi listaksi.

In [6]:
training_sequences = []
validation_sequences = []
attack_sequences = []

for file_name in os.listdir(TRAINING_DIR):
    with open(os.path.join(TRAINING_DIR,file_name)) as file:
        for line in file.readlines():
            training_sequences.append([int(syscall) for syscall in line.split()])
            break
            
print(f'Luettiin {len(training_sequences)} Training-datan jälkeä.')

for file_name in os.listdir(VALIDATION_DIR):
    with open(os.path.join(VALIDATION_DIR,file_name)) as file:
        for line in file.readlines():
            validation_sequences.append([int(syscall) for syscall in line.split()])
            break
            
print(f'Luettiin {len(validation_sequences)} Validation-datan jälkeä.')

for folder in os.listdir(ATTACK_DIR):
    attack_dir = os.path.join(ATTACK_DIR,folder)
    for file_name in os.listdir(attack_dir):
        with open(os.path.join(attack_dir,file_name)) as file:
            for line in file.readlines():
                attack_sequences.append([int(syscall) for syscall in line.split()])
                break
                
print(f'Luettiin {len(attack_sequences)} Attack-datan jälkeä.')

Luettiin 833 Training-datan jälkeä.
Luettiin 4372 Validation-datan jälkeä.
Luettiin 746 Attack-datan jälkeä.


### Datan muokkaus

Data ei kuitenkaan ole vielä tällaisenaan käyttökelpoista. Koska tavoitteena on muodostaa kategorinen harva matriisi, on ensin saata tietoa koko datajoukon uniikeista järjestelmäkutsuista.

In [7]:
unique_syscalls = set()

for sequence in training_sequences:
    for syscall in sequence:
        unique_syscalls.add(syscall)

for sequence in validation_sequences:
    for syscall in sequence:
        unique_syscalls.add(syscall)

for sequence in attack_sequences:
    for syscall in sequence:
        unique_syscalls.add(syscall)
            
print(f'Datajoukossa on {len(unique_syscalls)} uniikkia järjestelmäkutsua.')

Datajoukossa on 175 uniikkia järjestelmäkutsua.


Tehdään uniikeista kutsuista vielä `dictionary`, jossa uniikit arvot yhdistetään juoksevaan numerointiin. Tätä voidaan käyttää suoraan harvan matriisin tekoon siten, että sarakkeet saadaan menemään oikein.

In [8]:
syscalls_dict = {syscall:number for number,syscall in enumerate(unique_syscalls)}

print('Jotain järjestelmäkutsu-numero-pareja:')
for i,item in enumerate(syscalls_dict.items()):
    print(f'  Syscall {item[0]}\tNumero {item[1]}')
    if i == 9:
        print(f'  ...\t\t...')
        break
        

Jotain järjestelmäkutsu-numero-pareja:
  Syscall 1	Numero 0
  Syscall 3	Numero 1
  Syscall 4	Numero 2
  Syscall 5	Numero 3
  Syscall 6	Numero 4
  Syscall 7	Numero 5
  Syscall 8	Numero 6
  Syscall 9	Numero 7
  Syscall 10	Numero 8
  Syscall 11	Numero 9
  ...		...


Seuraavaksi on vuorossa jokaisen sekvenssin muuntaminen harvaksi matriisiksi. Tarkastellaan ensin yksittäistä sekvenssiä ja sen muuntamista hyödyntämällä suoraan edellisen koodiblokin viimeistä `sequence` muuttujaa.

In [9]:
import numpy as np

syscalls = np.zeros(shape=(len(sequence),len(unique_syscalls)),dtype=int)
for i, syscall in enumerate(sequence):
    syscalls[i,syscalls_dict[syscall]] = 1
syscalls

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

Tehdään sama seuraavaksi jokaiselle sekvenssille.

In [10]:
for i, sequence in enumerate(training_sequences):
    syscalls = np.zeros(shape=(len(sequence),len(unique_syscalls)),dtype=int)
    for j, syscall in enumerate(sequence):
        syscalls[j,syscalls_dict[syscall]] = 1
    training_sequences[i] = syscalls
    
for i, sequence in enumerate(validation_sequences):
    syscalls = np.zeros(shape=(len(sequence),len(unique_syscalls)),dtype=int)
    for j, syscall in enumerate(sequence):
        syscalls[j,syscalls_dict[syscall]] = 1
    validation_sequences[i] = syscalls
    
for i, sequence in enumerate(attack_sequences):
    syscalls = np.zeros(shape=(len(sequence),len(unique_syscalls)),dtype=int)
    for j, syscall in enumerate(sequence):
        syscalls[j,syscalls_dict[syscall]] = 1
    attack_sequences[i] = syscalls

Kurkataanpa vielä jokaisen osajoukon sisälle.

In [11]:
print(f'Training-joukon ensimmäinen jälki')
print(f'  koko:{training_sequences[0].shape}')

print(f'Validation-joukon ensimmäinen jälki')
print(f'  koko:{validation_sequences[0].shape}')

print(f'Attack-joukon ensimmäinen jälki')
print(f'  koko:{attack_sequences[0].shape}')


Training-joukon ensimmäinen jälki
  koko:(1986, 175)
Validation-joukon ensimmäinen jälki
  koko:(343, 175)
Attack-joukon ensimmäinen jälki
  koko:(334, 175)


### Nimiöity datajoukko

Lopuksi kullekin datajoukolle annetaan oikea leima. Luokka `1` tarkoittaa normaalia, `0` poikkeamaa. Näin malli oppii tuottamaan mahdollisimman tarkan tuloksen normaalitilalle.

In [12]:
training_sequences_labels = np.ones(shape=(len(training_sequences),),dtype=int).tolist()
validation_sequences_labels = np.ones(shape=(len(validation_sequences),),dtype=int).tolist()
attack_sequences_labels = np.zeros(shape=(len(attack_sequences),),dtype=int).tolist()

training_sequences_labeled = [training_sequences,training_sequences_labels]
validation_sequences_labeled = [validation_sequences, validation_sequences_labels]
attack_sequences_labeled = [attack_sequences, attack_sequences_labels]

print(f'Nimiöidyn Training-joukon ensimmäinen jälki')
print(f'  koko:{training_sequences_labeled[0][0].shape}')
print(f'  luokka:{training_sequences_labeled[1][0]}')

print(f'Nimiöidyn Validation-joukon ensimmäinen jälki')
print(f'  koko:{validation_sequences_labeled[0][0].shape}')
print(f'  luokka:{validation_sequences_labeled[1][0]}')

print(f'Nimiöidyn Attack-joukon ensimmäinen jälki')
print(f'  koko:{attack_sequences_labeled[0][0].shape}')
print(f'  luokka:{attack_sequences_labeled[1][0]}')

Nimiöidyn Training-joukon ensimmäinen jälki
  koko:(1986, 175)
  luokka:1
Nimiöidyn Validation-joukon ensimmäinen jälki
  koko:(343, 175)
  luokka:1
Nimiöidyn Attack-joukon ensimmäinen jälki
  koko:(334, 175)
  luokka:0


## Mallin rakentaminen

Kun data on kutakuinkin hyödynnettävässä muodossa, on aika siirtyä mallin pariin. Malli rakennetaan PyTorch-kehyksellä hyödyntäen `fastai`-opetuskirjastoa. Tärkeimmässä roolissa tulee olemaan oikeanlaisen `Dataset`-luokan rakentaminen, joka tarjoaa datan PyTorchin LSTM-implementaation vaatimassa muodossa.

### Dataset-luokka

Koska mallinnuksen keskipiisteessä on pohjimmiltaan ohjaamattoman oppimisen ongelma, jossa tavoitteena on oppia perustila poikkeamien erottamiseksi, koulutetaan malli erikseen sitä varten tarkoitetulla Training-joukolla. Opittua mallia arvioidaan käyttämällä Attack- ja Validation-joukkojen dataa.

In [13]:
from torch.utils.data import Dataset

class ADFATrain(Dataset):
    pass

class ADFAValidate(Dataset):
    pass